### 图神经网络环境配置，参见 https://www.bilibili.com/video/BV1184y1x71H?p=11

In [1]:
import os
import numpy as np
import pandas as pd
import os
import pandas as pd
from scipy.io import loadmat
import networkx as nx
import matplotlib.pyplot as plt
import torch
from torch_geometric.data import InMemoryDataset, Data
from tqdm import tqdm
from torch_geometric.loader import DataLoader
from sklearn.metrics import precision_score, f1_score, recall_score, accuracy_score
from torch_geometric.nn import TopKPooling,SAGEConv, GCNConv, SAGPooling
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as  gmp
from torch.nn import Linear,BatchNorm1d, ReLU
import torch.nn.functional as F
from imblearn.under_sampling import RandomUnderSampler

device = torch.device("mps" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

## 参数

In [2]:
epoch_num = 30  # 训练轮数
lr = 0.001      # 学习率
bs = 64         # 批次大小
isCause = False # 是否加入因果关系

dataset_root = '../Data' # 数据存放目录

## 因果关系的加入
注意：节点必须从0开始标号

In [3]:
def get_edge_index_and_edge_attr(ex_type):
    # source_nodes->target_nodes对应着一条有向边，而edge_attr为此条有向边的权值,无向图代表两个方向都有边
    source_nodes = [0, 1, 0, 4, 0, 7, 1, 2, 2, 3, 4, 5, 5, 6, 7, 8, 8, 9, 8, 14, 8, 19, 9, 10, 10, 11, 11, 12, 12, 13, 14, 15, 15, 16, 16, 17, 17, 18, 19, 20, 20, 21]
    target_nodes = [1, 0, 4, 0, 7, 0, 2, 1, 3, 2, 5, 4, 6, 5, 8, 7, 9, 8, 14, 8, 19, 8, 10, 9, 11, 10, 12, 11, 13, 12, 15, 14, 16, 15, 17, 16, 18, 17, 20, 19, 21, 20]
    edge_attr = None
    if isCause:
        # one leg stand
        if ex_type == 1:
            source_nodes = [0, 1, 2, 4, 6, 7, 8, 8 , 8 , 9, 10, 11, 12, 15, 15, 16, 17, 19, 19, 21]
            target_nodes = [4, 0, 3, 5, 5, 0, 7, 9, 14, 10, 11, 12, 13, 14, 16, 17, 18, 8 , 20, 20]
            edge_attr = None
    edge_index = torch.tensor([source_nodes, target_nodes], dtype=torch.long)
    return edge_index, edge_attr

## 数据处理部分
注意：如果图结构发生改变，请删除数据根目录/processed/下的缓存文件，以重新进行数据处理

In [4]:
# 这个类继承自InMemoryDataset，专门处理中小型数据集
class BinaryDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []
    # 如果根目录下的processed文件夹下已经尤处理好的图数据，则直接读取
    @property
    def processed_file_names(self):
        if isCause:
            return ['cause.dataset']
        else:
            return ['base.dataset',]
    def download(self):
        # Download to `self.raw_dir`.
        pass

    def process(self):
        """
        Load Data Seperate All data in Protective and Non-Prtective Data
        """
        # Assuming the 'Data' folder is in the current working directory
        data_folders = [dataset_root]

        # Initialize lists to store the separated dataframes
        protective_dfs = []
        non_protective_dfs = []

        # Loop through each data folder
        for data_folder in data_folders:
            # List all .mat files in the current data folder
            mat_files = [f for f in os.listdir(data_folder) if f.endswith('.mat')]
            # Load each mat file
            for mat_file in mat_files:
                # Construct the full path to the .mat file
                mat_path = os.path.join(data_folder, mat_file)
                # Load the .mat file
                mat_data = loadmat(mat_path)
                # Convert the data into a pandas dataframe
                df = pd.DataFrame(mat_data['data'])
                # Select only the first 70 columns and the last column (73rd) which contains the behavior label
                df = df.iloc[:, list(range(66)) + [70] + [72]]
                # Split the data based on the protective behavior label
                # Assuming the last column in df is the protective behavior label
                protective_behavior = df.iloc[:, -1]
                protective_df = df[protective_behavior == 1]
                non_protective_df = df[protective_behavior == 0]
                # Append the resulting dataframes to their respective lists
                protective_dfs.append(protective_df)
                non_protective_dfs.append(non_protective_df)

        # Concatenate all protective and non-protective dataframes
        all_protective_data = pd.concat(protective_dfs, axis=0, ignore_index=True)
        all_non_protective_data = pd.concat(non_protective_dfs, axis=0, ignore_index=True)

        # Now `all_protective_data` and `all_non_protective_data` hold the protective and non-protective data respectively
        # You can process these dataframes as needed for your analysis or save them to new .mat files
        # 保护和非保护数据个数 (77298, 68) (437247, 68)
        # 数据合并
        all_data = np.concatenate([all_protective_data,all_non_protective_data],axis=0)
        # 提取运动类型和保护性行为列
        ys = all_data[:,-2:]
        # 将特征列构建为N*V*C的形式，其中N=图个数（一条数据就可以看作一个图），V代表节点个数， C代表每个节点的特征个数
        x = all_data[:,np.newaxis,[0,22,44]]
        for node in range(2, 23):
            x_index = node - 1
            y_index = x_index + 22
            z_index = x_index + 44
            temp = all_data[:,np.newaxis,[x_index,y_index,z_index]]
            x = np.concatenate([x,temp], axis=1)
        # 将边整理为GNN要求的格式，边索引必须从0开始
       
        # 构建数据列表
        data_list = []
        # 一条数据构建为一个图
        for i in range(x.shape[0]):
            y = torch.tensor([ys[i,-1]], dtype=torch.float)   
            # 根据数据所属运动类型获得其拓扑结构
            edge_index, edge_attr = get_edge_index_and_edge_attr(ys[i,-2])
            if ys[i,-2] == 1:
                data = Data(x=torch.tensor(x[i], dtype=torch.float), edge_index=edge_index, edge_attr=edge_attr, y=y)
                data_list.append(data)
        # 将处理后的数据存储至指定根目录下
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])
            
        
# 加载并分割数据集      
def load_data_and_split(dataset_root):
    # 数据打乱
    dataset = BinaryDataset(root=dataset_root)
    # 随机打乱
    dataset = dataset.shuffle()
    # 数据划分逻辑
    num_graphs = len(dataset)
    train_size = int(num_graphs * 0.8)
    # train_size = int(num_graphs * 0.01)
    
    # 创建训练集和测试集的子集索引
    train_indices = list(range(train_size))
    test_indices = list(range(train_size, num_graphs))
    
    # 使用torch_geometric.data.Subset来划分数据
    train_dataset = dataset.index_select(train_indices)
    test_dataset = dataset.index_select(test_indices)
    print("isCause:",isCause)
    print("len of train_dataset:",len(train_dataset))
    print("len of test_dataset :",len(test_dataset))
    # 创建DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=False)
    
    return train_loader, test_loader

train_loader, test_loader =  load_data_and_split(dataset_root)

Processing...


isCause: False
len of train_dataset: 48786
len of test_dataset : 12197


Done!


## 模型定义
模块参见文档 https://pytorch-geometric.readthedocs.io/en/latest/

In [5]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = GCNConv(3,64)
        self.pool1 = TopKPooling(64, ratio=0.8)
        self.conv2 = GCNConv(64,64)
        self.pool2 = TopKPooling(64, ratio=0.8)
        self.conv3 = GCNConv(64,64)
        self.pool3 = TopKPooling(64, ratio=0.8)
        self.lin1 = Linear(64,64)
        self.lin2 = Linear(64,32)
        self.lin3 = Linear(32,1)
        # self.bn1 = BatchNorm1d(64)
        # self.bn2 = BatchNorm1d(32)
        self.act1 = ReLU()
        self.act2 = ReLU()

    
    def forward(self, data):
        x, edge_index, batch, edge_attr = data.x, data.edge_index, data.batch, data.edge_attr

        x = x.clone().detach().requires_grad_(True)
        x = F.relu(self.conv1(x, edge_index, edge_attr))
        # x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, edge_attr, batch)
        x1 = gap(x, batch)
        
        x = F.relu(self.conv2(x, edge_index, edge_attr))
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, edge_attr, batch)
        x2 = gap(x, batch)
        
        # x = F.relu(self.conv3(x, edge_index))
        # x, edge_index, _, batch, _, _ = self.pool3(x, edge_index, None, batch)
        # x3 = gap(x, batch)  
        
        x = x1 + x2
        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)
        x = F.dropout(x, p=0.5, training=self.training)
        
        # 输出值为0-1之间 （0为非保护 1为保护）
        x = torch.sigmoid(self.lin3(x)).squeeze(1)
        return x     

## 模型训练和测试

In [6]:

def train(train_loader):
    model.train()
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data)
        label = data.y.to(device)
        loss = crit(output, label)
        loss.backward()
        optimizer.step()
y_true = []
y_pred = []
def test(loader):
    model.eval()


    with torch.no_grad():  # 在评估模式下不计算梯度
        for data in loader:
            data = data.to(device)
            out = model(data)  # 前向传播
            pred = (out >= 0.5).int()

            y_true.extend(data.y.clone().detach().tolist())  # 收集真实标签
            y_pred.extend(pred.clone().detach().tolist())  # 收集预测标签
    
    accuracy = accuracy_score(y_true, y_pred)
    macro_precision = precision_score(y_true, y_pred,  pos_label=1, average='macro') 
    
    f1 = f1_score(y_true, y_pred, pos_label=1)  # Only report results for the class specified by pos_label. 考虑了非平衡
    recall = recall_score(y_true, y_pred, pos_label=1)
    return accuracy, macro_precision, f1, recall

model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
crit = torch.nn.BCELoss()
for epoch in range(1, epoch_num):
    train(train_loader)
    train_accuracy, train_macro_precision, train_f1, train_recall = test(train_loader)
    test_accuracy,  test_macro_precision,  test_f1 , test_recall  = test(test_loader)
    
    print(f'Epoch: {epoch:03d}  -------------------------- '
          f'\nTrain: accuracy: {train_accuracy:.4f}, Macro Precision: {train_macro_precision:.4f}, F1: {train_f1:.4f}, recall: {train_recall:.4f}  '
          f'\nTest : accuracy: {test_accuracy:.4f}, Macro Precision: {test_macro_precision:.4f}, F1: {test_f1:.4f}, recall: {test_recall:.4f}')

Epoch: 001  -------------------------- 
Train: accuracy: 0.8789, Macro Precision: 0.8935, F1: 0.6675, recall: 0.5259  
Test : accuracy: 0.8786, Macro Precision: 0.8933, F1: 0.6661, recall: 0.5242
Epoch: 002  -------------------------- 
Train: accuracy: 0.8812, Macro Precision: 0.8873, F1: 0.6812, recall: 0.5496  
Test : accuracy: 0.8812, Macro Precision: 0.8867, F1: 0.6821, recall: 0.5513
Epoch: 003  -------------------------- 
Train: accuracy: 0.8804, Macro Precision: 0.8875, F1: 0.6779, recall: 0.5447  
Test : accuracy: 0.8801, Macro Precision: 0.8875, F1: 0.6766, recall: 0.5429
Epoch: 004  -------------------------- 
Train: accuracy: 0.8800, Macro Precision: 0.8881, F1: 0.6757, recall: 0.5411  
Test : accuracy: 0.8798, Macro Precision: 0.8881, F1: 0.6751, recall: 0.5402
Epoch: 005  -------------------------- 
Train: accuracy: 0.8821, Macro Precision: 0.8862, F1: 0.6859, recall: 0.5572  
Test : accuracy: 0.8823, Macro Precision: 0.8856, F1: 0.6874, recall: 0.5598
Epoch: 006  --------

In [ ]:
import matplotlib.pyplot as plt

# Data for each metric by epoch
epochs = range(1, 30)

#Base
train_accuracy_base = [0.8771, 0.8760, 0.8787, 0.9007, 0.9044, 0.9071, 0.9142, 0.9238, 0.9294, 0.9124, 0.9133, 0.9376, 0.9387, 0.9372, 0.9360, 0.9200, 0.9386, 0.9389, 0.9326, 0.9003, 0.9328, 0.9377, 0.9402, 0.9256, 0.9198, 0.9275, 0.9227, 0.9266, 0.9300]
test_accuracy_base= [0.8806, 0.8796, 0.8815, 0.9051, 0.9056, 0.9103, 0.9170, 0.9260, 0.9314, 0.9156, 0.9165, 0.9403, 0.9412, 0.9401, 0.9369, 0.9251, 0.9411, 0.9422, 0.9319, 0.9057, 0.9325, 0.9431, 0.9435, 0.9289, 0.9206, 0.9283, 0.9252, 0.9287, 0.9329]
train_precision_base = [0.8879, 0.8912, 0.8867, 0.8745, 0.8786, 0.8599, 0.8779, 0.8911, 0.8903, 0.8654, 0.8640, 0.9001, 0.8987, 0.8985, 0.9020, 0.8846, 0.9008, 0.8993, 0.8967, 0.8490, 0.8916, 0.9019, 0.9018, 0.8796, 0.8884, 0.8869, 0.8861, 0.8850, 0.8910]
test_precision_base = [0.8954, 0.8989, 0.8931, 0.8811, 0.8803, 0.8633, 0.8818, 0.8953, 0.8925, 0.8692, 0.8671, 0.9037, 0.9017, 0.9020, 0.9031, 0.8914, 0.9044, 0.9033, 0.8957, 0.8549, 0.8907, 0.9094, 0.9057, 0.8833, 0.8892, 0.8879, 0.8899, 0.8875, 0.8954]
train_f1_base = [0.6643, 0.6573, 0.6722, 0.7696, 0.7797, 0.8141, 0.8161, 0.8369, 0.8565, 0.8271, 0.8376, 0.8738, 0.8788, 0.8740, 0.8674, 0.8299, 0.8763, 0.8787, 0.8610, 0.8101, 0.8668, 0.8725, 0.8806, 0.8577, 0.8255, 0.8538, 0.8378, 0.8528, 0.8578]
test_f1_base  = [0.6707, 0.6642, 0.6761, 0.7780, 0.7805, 0.8194, 0.8204, 0.8392, 0.8593, 0.8317, 0.8420, 0.8780, 0.8826, 0.8787, 0.8678, 0.8392, 0.8797, 0.8843, 0.8579, 0.8180, 0.8652, 0.8822, 0.8862, 0.8626, 0.8261, 0.8538, 0.8411, 0.8554, 0.8617]
train_recall_base = [0.5255, 0.5139, 0.5373, 0.7169, 0.7311, 0.8790, 0.8230, 0.8446, 0.9102, 0.9061, 0.9664, 0.9339, 0.9602, 0.9417, 0.9037, 0.8433, 0.9406, 0.9573, 0.9017, 0.9183, 0.9447, 0.9218, 0.9529, 0.9686, 0.8204, 0.9144, 0.8627, 0.9187, 0.9120]
test_recall_base  = [0.5293, 0.5182, 0.5382, 0.7234, 0.7302, 0.8854, 0.8248, 0.8401, 0.9122, 0.9072, 0.9690, 0.9350, 0.9615, 0.9450, 0.9019, 0.8512, 0.9383, 0.9611, 0.8954, 0.9226, 0.9429, 0.9276, 0.9575, 0.9711, 0.8205, 0.9108, 0.8612, 0.9183, 0.9097]

#CGCN
train_accuracy_cgcn = [0.8919, 0.8781, 0.8947, 0.8872, 0.9076, 0.9318, 0.9307, 0.9367, 0.9347, 0.9360, 0.9377, 0.9305, 0.9372, 0.9367, 0.9389, 0.9368, 0.9370, 0.9353, 0.9388, 0.9324, 0.9365, 0.9377, 0.9397, 0.9338, 0.9318, 0.9395, 0.9404, 0.9413, 0.9407]
test_accuracy_cgcn  = [0.8922, 0.8765, 0.8946, 0.8863, 0.9069, 0.9323, 0.9303, 0.9384, 0.9344, 0.9384, 0.9397, 0.9330, 0.9387, 0.9381, 0.9412, 0.9391, 0.9398, 0.9373, 0.9406, 0.9336, 0.9400, 0.9383, 0.9420, 0.9368, 0.9341, 0.9410, 0.9418, 0.9426, 0.9417]
train_precision_cgcn = [0.8772, 0.8898, 0.8799, 0.8829, 0.8881, 0.8941, 0.8874, 0.8960, 0.8930, 0.8954, 0.8980, 0.8933, 0.8973, 0.8983, 0.9005, 0.8967, 0.8970, 0.8935, 0.8974, 0.8876, 0.8947, 0.8956, 0.8985, 0.8962, 0.8950, 0.9014, 0.8999, 0.9011, 0.9003]
test_precision_cgcn  = [0.8779, 0.8880, 0.8802, 0.8813, 0.8875, 0.8960, 0.8884, 0.8994, 0.8937, 0.8996, 0.9019, 0.8978, 0.9002, 0.9016, 0.9046, 0.9010, 0.9017, 0.8969, 0.9011, 0.8903, 0.9005, 0.8974, 0.9026, 0.9015, 0.8990, 0.9045, 0.9031, 0.9041, 0.9029]
train_f1_cgcn = [0.7316, 0.6655, 0.7405, 0.7085, 0.7821, 0.8599, 0.8636, 0.8744, 0.8708, 0.8727, 0.8753, 0.8563, 0.8744, 0.8721, 0.8768, 0.8739, 0.8741, 0.8724, 0.8799, 0.8702, 0.8750, 0.8782, 0.8817, 0.8643, 0.8591, 0.8779, 0.8822, 0.8841, 0.8830]
test_f1_cgcn  = [0.7369, 0.6659, 0.7446, 0.7110, 0.7840, 0.8622, 0.8639, 0.8788, 0.8717, 0.8786, 0.8805, 0.8630, 0.8788, 0.8757, 0.8828, 0.8794, 0.8811, 0.8777, 0.8842, 0.8736, 0.8827, 0.8805, 0.8872, 0.8717, 0.8653, 0.8822, 0.8860, 0.8876, 0.8858]
train_recall_cgcn = [0.6393, 0.5260, 0.6517, 0.5944, 0.7197, 0.9075, 0.9522, 0.9554, 0.9548, 0.9509, 0.9492, 0.8984, 0.9486, 0.9354, 0.9437, 0.9496, 0.9493, 0.9592, 0.9721, 0.9828, 0.9643, 0.9740, 0.9743, 0.9152, 0.9018, 0.9441, 0.9690, 0.9713, 0.9704]
test_recall_cgcn  = [0.6470, 0.5272, 0.6579, 0.5992, 0.7236, 0.9080, 0.9473, 0.9561, 0.9543, 0.9547, 0.9515, 0.9038, 0.9529, 0.9340, 0.9484, 0.9515, 0.9550, 0.9645, 0.9712, 0.9828, 0.9677, 0.9740, 0.9768, 0.9199, 0.9073, 0.9466, 0.9687, 0.9705, 0.9687]

# Plotting
fig, axs = plt.subplots(2, 2, figsize=(14, 10))
axs = axs.ravel()

# Accuracy plot
axs[0].plot(epochs, train_accuracy_base, label='Train Accuracy-Baseline')
axs[0].plot(epochs, test_accuracy_base, label='Test Accuracy-Baseline')
axs[0].plot(epochs, train_accuracy_cgcn, label='Train Accuracy-Causal GCN')
axs[0].plot(epochs, test_accuracy_cgcn, label='Test Accuracy-Causal GCN')
axs[0].set_title('Accuracy')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Accuracy')
axs[0].legend()

# Macro Precision plot
axs[1].plot(epochs, train_precision_base, label='Train Macro Precision-Baseline')
axs[1].plot(epochs, test_precision_base, label='Test Macro Precision-Baseline')
axs[1].plot(epochs, train_precision_cgcn, label='Train Macro Precision-Causal GCN')
axs[1].plot(epochs, test_precision_cgcn, label='Test Macro Precision-Causal GCN')
axs[1].set_title('Macro Precision')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Macro Precision')
axs[1].legend()

# F1 Score plot
axs[2].plot(epochs, train_f1_base, label='Train F1-Baseline')
axs[2].plot(epochs, test_f1_base, label='Test F1-Baseline')
axs[2].plot(epochs, train_f1_cgcn, label='Train F1-Causal GCN')
axs[2].plot(epochs, test_f1_cgcn, label='Test F1-Causal GCN')
axs[2].set_title('F1 Score')
axs[2].set_xlabel('Epoch')
axs[2].set_ylabel('F1 Score')
axs[2].legend()

# Recall plot
axs[3].plot(epochs, train_recall_base, label='Train Recall-Baseline')
axs[3].plot(epochs, test_recall_base, label='Test Recall-Baseline')
axs[3].plot(epochs, train_recall_cgcn, label='Train Recall-Causal GCN')
axs[3].plot(epochs, test_recall_cgcn, label='Test Recall-Causal GCN')
axs[3].set_title('Recall')
axs[3].set_xlabel('Epoch')
axs[3].set_ylabel('Recall')
axs[3].legend()

plt.tight_layout()
plt.show()
